# Assignment 1 : Decision Tress using `scikit-learn`

Scikit-learn provides a range of supervised and unsupervised learning algorithms via a consistent interface in Python. In this assigment you'll explore how to train decision trees using the `scikit-learn` library. The scikit-learn documentation can be found [here](http://scikit-learn.org/stable/documentation.html).

In this assignment we'll attempt to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains 1151 instances and 20 attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

In [1]:
# You may add additional imports
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import time

%matplotlib inline

In [2]:
# Read the data from csv file
col_names = []
for i in range(20):
    if i == 0:
        col_names.append('quality')
    if i == 1:
        col_names.append('prescreen')
    if i >= 2 and i <= 7:
        col_names.append('ma' + str(i))
    if i >= 8 and i <= 15:
        col_names.append('exudate' + str(i))
    if i == 16:
        col_names.append('euDist')
    if i == 17:
        col_names.append('diameter')
    if i == 18:
        col_names.append('amfm_class')
    if i == 19:
        col_names.append('label')

data = pd.read_csv("messidor_features.txt", names = col_names)
print(data.shape)
data.head(10)

(1151, 20)


quality  prescreen  ma2  ma3  ma4  ma5  ma6  ma7   exudate8   exudate9  \
0        1          1   22   22   22   19   18   14  49.895756  17.775994   
1        1          1   24   24   22   18   16   13  57.709936  23.799994   
2        1          1   62   60   59   54   47   33  55.831441  27.993933   
3        1          1   55   53   53   50   43   31  40.467228  18.445954   
4        1          1   44   44   44   41   39   27  18.026254   8.570709   
5        1          1   44   43   41   41   37   29  28.356400   6.935636   
6        1          0   29   29   29   27   25   16  15.448398   9.113819   
7        1          1    6    6    6    6    2    1  20.679649   9.497786   
8        1          1   22   21   18   15   13   10  66.691933  23.545543   
9        1          1   79   75   73   71   64   47  22.141784  10.054384   

   exudate10  exudate11  exudate12  exudate13  exudate14  exudate15    euDist  \
0   5.270920   0.771761   0.018632   0.006864   0.003923   0.003923  0.486903   
1   3.325423   0.234185   0.003903   0.003903   0.003903   0.003903  0.520908   
2  12.687485   4.852282   1.393889   0.373252   0.041817   0.007744  0.530904   
3   9.118901   3.079428   0.840261   0.272434   0.007653   0.001531  0.483284   
4   0.410381   0.000000   0.000000   0.000000   0.000000   0.000000  0.475935   
5   2.305771   0.323724   0.000000   0.000000   0.000000   0.000000  0.502831   
6   1.633493   0.000000   0.000000   0.000000   0.000000   0.000000  0.541743   
7   1.223660   0.150382   0.000000   0.000000   0.000000   0.000000  0.576318   
8   6.151117   0.496372   0.000000   0.000000   0.000000   0.000000  0.500073   
9   0.874633   0.099780   0.023386   0.000000   0.000000   0.000000  0.560959   

   diameter  amfm_class  label  
0  0.100025           1      0  
1  0.144414           0      0  
2  0.128548           0      1  
3  0.114790           0      0  
4  0.123572           0      1  
5  0.126741           0      1  
6  0.139575           0      1  
7  0.071071           1      0  
8  0.116793           0      1  
9  0.109134           0      1

### 1. Data preprocessing  & dimensionality reduction with PCA

Q1. Separate the feature columns from the class label column. You should end up with two separate data frames - one that contains all of the feature values and one that contains the class labels.

In [12]:
data_y = data['label']
data_x = data.drop(['label'], axis=1)
data_x.head()

quality  prescreen  ma2  ma3  ma4  ma5  ma6  ma7   exudate8   exudate9  \
0        1          1   22   22   22   19   18   14  49.895756  17.775994   
1        1          1   24   24   22   18   16   13  57.709936  23.799994   
2        1          1   62   60   59   54   47   33  55.831441  27.993933   
3        1          1   55   53   53   50   43   31  40.467228  18.445954   
4        1          1   44   44   44   41   39   27  18.026254   8.570709   

   exudate10  exudate11  exudate12  exudate13  exudate14  exudate15    euDist  \
0   5.270920   0.771761   0.018632   0.006864   0.003923   0.003923  0.486903   
1   3.325423   0.234185   0.003903   0.003903   0.003903   0.003903  0.520908   
2  12.687485   4.852282   1.393889   0.373252   0.041817   0.007744  0.530904   
3   9.118901   3.079428   0.840261   0.272434   0.007653   0.001531  0.483284   
4   0.410381   0.000000   0.000000   0.000000   0.000000   0.000000  0.475935   

   diameter  amfm_class  
0  0.100025           1  
1  0.144414           0  
2  0.128548           0  
3  0.114790           0  
4  0.123572           0

Q3 . Split your dataset into training and test sets (80% - 20% split). Use `sklearn.model_selection.train_test_split` to help you in this task. You should be working with your standardized features from here forward. Display how many records are in the training set and how many are in the test set.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.20, random_state=None)
print("X_train=",X_train.shape, " y_train=", y_train.shape)
print("X_test=",X_test.shape, " y_test=", y_test.shape)

X_train= (920, 19)  y_train= (920,)
X_test= (231, 19)  y_test= (231,)


Q2. Use `sklearn.preprocessing.StandardScaler` to standardize the dataset’s features (mean = 0 and variance = 1). 

Only standardize the the features, not the class labels! This will be required for running the principal component analysis (PCA) dimensionality reduction later. Note that `StandardScaler` returns a numpy array.

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Q4. PCA is affected by the scale of the features that is why it is important to standardize the dataset first. The principle components generated by PCA are sensitive to the shape of the data in d-dimensional space. 
* Carry out a principal components analysis using `sklearn.decomposition.PCA` Note that you are fitting PCA on the **training set** only (NOT the test set).
* Use the `pca.explained_variance_ratio_` field to determine how many principal components are needed so that 95% variance is retained. 
* Reduce the PCA-transformed-dataset to this number of columns and you'll use the resulting dataset for subsequent tasks.

* Once the training set's dimensionality has been reduced with PCA, transform the **test set** to the principal component space that was created. (Do not fit a new PCA. Use the same one that was created with the training set.)

In [15]:
from sklearn.decomposition import PCA 

pca = PCA()
pca_data = pca.fit_transform(X_train_scaled)
pca_data_df = pd.DataFrame(pca_data)
pca_data_df.head()

0         1         2         3         4         5         6  \
0  0.806962  1.940751 -0.154704 -0.465379  0.007526  0.189223 -0.263091   
1 -2.451153 -0.593782  0.381228 -1.201078  0.205777  0.261467 -0.401937   
2 -2.410752  1.107564 -1.193278 -0.195863  0.451803  0.591102 -2.535728   
3 -2.821252 -0.428638  0.239674 -0.305404  0.154942 -0.261214 -0.108793   
4 -2.410800  0.180069 -0.649936  0.977911 -0.243603 -0.917547  0.262705   

          7         8         9        10        11        12        13  \
0 -1.109573  1.003504  0.053484  0.128840  0.078410 -0.093881  0.285828   
1  0.829241  0.546137 -0.386757  0.109243  0.168253 -0.073481 -0.002716   
2 -1.109991 -0.669887  1.860845 -0.581167  0.190731 -0.125080 -0.075676   
3  0.828622  0.310010 -0.144118 -0.106573  0.127964 -0.092594 -0.131273   
4 -1.146058 -0.833668 -0.301269 -0.060206 -0.958734  0.316378  0.243241   

         14        15        16        17        18  
0 -0.237407 -0.239505  0.070086 -0.025878  0.012153  
1  0.048172 -0.049978 -0.035828  0.012977  0.036141  
2  0.147427  0.088296  0.073844 -0.018236 -0.032476  
3 -0.031165  0.049489 -0.050084 -0.001591  0.014283  
4  0.053086 -0.060646 -0.004128  0.018686  0.013801

In [88]:
var_exp = pca.explained_variance_ratio_
cum_var_exp = np.cumsum(var_exp)
n_cols = 1 + np.argmax(cum_var_exp > 0.95)
print("collumns to keep:", n_cols)
X_train_pca = pca_data[:, :n_cols]

collumns to keep: 9


In [87]:
X_test_pca = pca.transform(X_test_scaled)[:, :n_cols] 
print (X_train_pca.shape, X_test_pca.shape) 

(920, 9) (231, 9)


### 2. Training Decision Trees in `scikit-learn`

Q5. Use `sklearn.tree.DecisionTreeClassifier` to fit a decision tree classifier on the training set. Use entropy as the split criterion. 

In [75]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion='entropy')
tree = tree.fit(X_train_pca, y_train)
tree

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

Q6. Now that the tree has been learned from the training data, we can run the test data through and predict classes for our test data. Use the `predict` method of `DecisionTreeClassifier` to classify the test data. Then use `sklearn.metrics.accuracy_score` to print out the accuracy of the classifier on the test set.

In [76]:
from sklearn.metrics import accuracy_score

predicted_y = tree.predict(X_test_pca)
accuracy = accuracy_score(y_test, predicted_y, normalize=True)
print(accuracy)

0.6363636363636364


Q7. Note that the DecisionTree classifier has many parameters that can be set. Try tweaking parameters like split criterion, max_depth, min_impurity_decrease, min_samples_leaf, min_samples_split, etc. to see how they affect accuracy.

In [72]:
'''
max_depth : int or None, optional (default=None) 
    The maximum depth of the tree.
min_impurity_decrease : float, optional (default=0.)
    A node will be split if this split induces a decrease of the impurity greater than or equal to this value.
min_samples_leaf : int, float, optional (default=1) 
    The minimum number of samples required to be at a leaf node.
min_samples_split : int, float, optional (default=2)
    The minimum number of samples required to split an internal node:
'''

tree_2 = DecisionTreeClassifier(criterion='gini', max_depth=4, min_impurity_decrease=0, 
                                min_samples_leaf=3, min_samples_split=2)
tree_2 = tree_2.fit(X_train_pca, y_train)
predicted_y_2 = tree_2.predict(X_test_pca)
accuracy_2 = accuracy_score(y_test, predicted_y_2, normalize=True)
accuracy_2


0.6147186147186147

### 3. Using K-fold Cross Validation

You have now built a decision tree and tested it's accuracy using the "holdout" method. But as discussed in class, this is not sufficient for estimating generalization accuracy. Instead, we should use Cross Validation to get a better estimate of accuracy. You will do that next.

Q8. Use `sklearn.model_selection.cross_val_score` to perform 10-fold cross validation on your decision tree. Report the accuracy of the model. For this step, revert back to using the original data before it got scaled and underwent PCA. So you want to use the data (features and labels) that came out of Q1. 

In [77]:
from sklearn.model_selection import cross_val_score

tree_k= DecisionTreeClassifier(criterion='entropy')
scores = cross_val_score(tree_k, data_x, data_y, cv=10)
print('Scores: ', scores)
print('Average: ', scores.mean())

Scores:  [0.62931034 0.5826087  0.66086957 0.62608696 0.63478261 0.66086957
 0.62608696 0.5826087  0.68695652 0.5826087 ]
Average:  0.6272788605697152


Q9. Now we want to tune our model to use the best parameters to avoid overfitting to our training data. Grid search is an approach to parameter tuning that will methodically build and evaluate a model for each combination of algorithm parameters specified in a grid. 
* Use `sklearn.model_selection.GridSearchCV` to find the best `max_depth`, `max_features`, and `min_samples_leaf` for your tree. Use 'accuracy' for the scoring criteria.
* Try the values [5,10,15,20] for `max_depth` and `min_samples_leaf`. Try [5,10,15] for `max_features`. 
* Use a 5-fold cross validation. 
* Print out the best value for each of the tested parameters.
* Print out the accuracy of the model with these best values.

In [80]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth':[5,10,15,20], 'min_samples_leaf':[5,10,15,20], 'max_features':[5,10,15]}
gscv = GridSearchCV(tree_k, param_grid=parameters, scoring='accuracy', cv=5)
gscv.fit(data_x, data_y)
print(gscv.best_params_)
print(gscv.best_score_)

{'max_depth': 20, 'max_features': 5, 'min_samples_leaf': 20}
0.6507384882710686


Q10. To perform the nested cross-validation that we discussed in class, you'll now need to pass the `GridSearchCV` into a `cross_val_score`. 

What this does is: the `cross_val_score` splits the data in to train and test sets for the first fold, and it passes the train set into `GridSearchCV`. `GridSearchCV` then splits that set into train and validation sets for k number of folds (the inner CV loop). The hyper-parameters for which the average score over all inner iterations is best, is reported as the `best_params_`, `best_score_`, and `best_estimator_`(best decision tree). This best decision tree is then evaluated with the test set from the `cross_val_score` (the outer CV loop). And this whole thing is repeated for the remaining k folds of the `cross_val_score` (the outer CV loop). 

That is a lot of explanation for a very complex (but IMPORTANT) process, which can all be performed with a single line of code!

Be patient for this one to run. The nested cross-validation loop can take some time. An asterisk [\*] next to the cell indicates that it is still running.

Print the accuracy of your tuned, cross-validated model. This is the official accuracy that you would report for your model.

In [84]:
cross_val_score(gscv, data_x, data_y, cv=10).mean()

0.6316266866566715

Q11. Our accuracy rate isn't very good. We wouldn't want to use this model in the real world to actually diagnose patients because it is wrong about 40% of the time! What could you do to improve the accuracy of the model? Answer as a comment:

In [ ]:
'''
Recollect more data to train the model, use a method for feature selection and reduce dimensionality,
get rid of higly correlated features, post-prune the tree can also improve the accuracy I think. 
'''